In [1]:
!pip install timm


In [2]:
!pip install kaggle

# Setup Kaggle API for downloading the dataset
!mkdir ~/.kaggle
!cp /kaggle/input/kagglejson/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip dataset
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder
!mkdir -p /kaggle/working/car_data
!unzip -q stanford-car-dataset-by-classes-folder.zip -d /kaggle/working/car_data


cp: cannot stat '/kaggle/input/kagglejson/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder
License(s): other
100%|██████████████████████████████████████▉| 1.82G/1.83G [00:16<00:00, 154MB/s]
100%|███████████████████████████████████████| 1.83G/1.83G [00:16<00:00, 121MB/s]


In [3]:
# Import necessary libraries
from fastai.vision.all import *
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback
from fastai.metrics import F1Score, accuracy
from sklearn.model_selection import KFold
import copy
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
# Dataset paths
train_path = Path('/kaggle/working/car_data/car_data/car_data/train')
test_path = Path('/kaggle/working/car_data/car_data/car_data/test')

In [5]:
# Image transformations
item_tfms = Resize(460)
batch_tfms = [*aug_transforms(mult=2.0, do_flip=True, flip_vert=False, max_rotate=30., 
                              max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, 
                              p_lighting=0.75), RandomErasing(p=0.5), Normalize.from_stats(*imagenet_stats)]

# Create the DataLoader with a validation set split from the train set
dls = ImageDataLoaders.from_folder(
    Path('/kaggle/working/car_data/car_data/car_data'),
    train='train', valid_pct=0.2,  # Use 20% of training data for validation
    item_tfms=item_tfms, batch_tfms=batch_tfms, bs=16, seed=42
)

In [6]:
# Learner setup with ResNet-152
learn = cnn_learner(
    dls, 
    resnet152, 
    metrics=[accuracy, F1Score(average='weighted'),Precision(average='weighted')],  # Specify average='weighted' for multiclass
    pretrained=True,
    cbs=[MixUp(alpha=0.4)]  # Maintain MixUp effect
)

# Find learning rates (assuming you've previously found these with lr_find)
lr_min = 0.0019054606556892395
lr_steep = 7.585775847473997e-07
print(f"Suggested Learning Rates: min={lr_min}, steep={lr_steep}")

/usr/local/lib/python3.10/dist-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:01<00:00, 168MB/s]  


Suggested Learning Rates: min=0.0019054606556892395, steep=7.585775847473997e-07


In [7]:

# Fine-tune the model with the SaveModelCallback to save every epoch
learn.fine_tune(
    10,  # Number of epochs
    base_lr=lr_min,  # Use suggested minimum learning rate
    cbs=[
        SaveModelCallback(
            every_epoch=True, 
            with_opt=True,
            fname='model_epoch'  # Save model with the generic name pattern
        )
    ]
)

# After training finishes, save the final model with a .pkl extension
learn.export('/kaggle/working/final_model.pkl')

epoch,train_loss,valid_loss,accuracy,f1_score,precision_score,time
0,5.537552,4.558063,0.075687,0.058015,0.090906,08:59


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epoch,train_loss,valid_loss,accuracy,f1_score,precision_score,time
0,4.197004,2.582158,0.409330,0.384937,0.457747,11:58
1,3.312049,1.405081,0.663577,0.653636,0.723536,11:59
2,2.789615,0.835993,0.786531,0.784446,0.820858,11:58
3,2.493768,0.577964,0.848934,0.845735,0.867202,11:58
4,2.243516,0.495797,0.869323,0.865441,0.877839,11:58
5,1.996258,0.378701,0.903614,0.903211,0.914139,11:58
6,1.819704,0.324932,0.917207,0.917315,0.926263,11:58
7,1.649664,0.289713,0.925239,0.925477,0.931515,11:59
8,1.629172,0.279743,0.930491,0.930742,0.937721,11:58
9,1.668214,0.274681,0.930182,0.929911,0.936243,11:58


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

In [ ]:
# Generate predictions and true labels
preds, targs = learn.get_preds()  # Get model predictions and true labels

# Convert predictions to class indices
predicted_classes = preds.argmax(dim=1).numpy()

# True labels
true_classes = targs.numpy()

# Calculate precision, recall, F1 score, and accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')
accuracy = accuracy_score(true_classes, predicted_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Display confusion matrix using ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Precision: 0.9362
Recall: 0.9302
F1 Score: 0.9299
Accuracy: 0.9302
